In [1]:
import pandas as pd
import numpy as np

In [2]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('D:\\DS\\DS school\\boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [4]:
### Ваш код ###
from sklearn.model_selection import train_test_split

X = data.drop('MEDV',axis=1)
y = data['MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
### Ваш код ###
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score


# линейная модель
model_lr = LinearRegression()
model_lr.fit(X_train,y_train)
lr_pred = model_lr.predict(X_test)

# Ridge
model_ridge = Ridge()
model_ridge.fit(X_train,y_train)
ridge_pred = model_ridge.predict(X_test)

# Lasso
model_lasso = Lasso()
model_lasso.fit(X_train,y_train)
lasso_pred = model_lasso.predict(X_test)

r2_lr = r2_score(y_test, lr_pred)
r2_ridge = r2_score(y_test, ridge_pred)
r2_lasso = r2_score(y_test, lasso_pred)
results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]

3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [6]:
### Ваш код ###
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import GridSearchCV

params = [10**i for i in range(-5, 6)]

# RidgeCV
model_ridge_CV = RidgeCV(alphas = params)
model_ridge_CV.fit(X_train, y_train)
ridgeCV_pred = model_ridge_CV.predict(X_test)

# LassoCV
model_lasso_CV = LassoCV(alphas = params)
model_lasso_CV.fit(X_train, y_train)
lassoCV_pred = model_lasso_CV.predict(X_test)

#GridSearchCV
param_grid = {'alpha': params}
model_ridge_grid = Ridge()
model_lasso_grid = Lasso()
grid_model_ridge = GridSearchCV(estimator=model_ridge_grid, param_grid=param_grid, cv=5)
grid_model_ridge.fit(X_train, y_train)
grid_model_lasso = GridSearchCV(estimator=model_lasso_grid, param_grid=param_grid, cv=5)
grid_model_lasso.fit(X_train, y_train)


r2_ridge_grid_search = grid_model_ridge.best_score_
r2_ridge_cv = r2_score(y_test, ridgeCV_pred)
r2_lasso_grid_search = grid_model_lasso.best_score_
r2_lasso_cv = r2_score(y_test, lassoCV_pred)
results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]

r2_ridge_cv, r2_lasso_cv, r2_ridge_grid_search, r2_lasso_grid_search

(0.7034285096341584, 0.70349497581526, 0.712425401352967, 0.7123497706813428)

4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [7]:
### Ваш код ###
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.linear_model import RidgeCV, LassoCV

#StandardScaler
standart_scaler_x = StandardScaler()
standart_scaler_x.fit(X_train)
X_train_standart_scaler_transformed = standart_scaler_x.transform(X_train)
X_test_standart_scaler_transformed = standart_scaler_x.transform(X_test)

#MinMaxScaler
minmax_scaler_x = MinMaxScaler()
minmax_scaler_x.fit(X_train)
X_train_minmax_scaler_transformed = minmax_scaler_x.transform(X_train)
X_test_minmax_scaler_transformed = minmax_scaler_x.transform(X_test)

# Ridge
model_ridge_standart_scaler = Ridge()
model_ridge_standart_scaler.fit(X_train_standart_scaler_transformed,y_train)
ridge_standart_scaler_pred = model_ridge_standart_scaler.predict(X_test_standart_scaler_transformed)

model_ridge_minmax_scaler = Ridge()
model_ridge_minmax_scaler.fit(X_train_minmax_scaler_transformed,y_train)
ridge_minmax_scaler_pred = model_ridge_minmax_scaler.predict(X_test_minmax_scaler_transformed)

# Lasso
model_lasso_standart_scaler = Lasso()
model_lasso_standart_scaler.fit(X_train_standart_scaler_transformed,y_train)
lasso_standart_scaler_pred = model_lasso_standart_scaler.predict(X_test_standart_scaler_transformed)

model_lasso_minmax_scaler = Lasso()
model_lasso_minmax_scaler.fit(X_train_minmax_scaler_transformed,y_train)
lasso_minmax_scaler_pred = model_lasso_minmax_scaler.predict(X_test_minmax_scaler_transformed)

r2_ridge_standart_scaler = r2_score(y_test, ridge_standart_scaler_pred)
r2_ridge_min_max_scaler = r2_score(y_test, ridge_minmax_scaler_pred)
r2_lasso_standart_scaler = r2_score(y_test, lasso_standart_scaler_pred)
r2_lasso_min_max_scaler = r2_score(y_test, lasso_minmax_scaler_pred)
results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]

r2_ridge_standart_scaler, r2_ridge_min_max_scaler, r2_lasso_standart_scaler, r2_lasso_min_max_scaler

(0.7032633383504197, 0.7010203723781033, 0.6277052014931489, 0.197922380429706)

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [8]:
### Ваш код ###
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import r2_score

#StandardScaler
standart_scaler_x = StandardScaler()
standart_scaler_x.fit(X_train)
X_train_standart_scaler_transformed = standart_scaler_x.transform(X_train)
X_test_standart_scaler_transformed = standart_scaler_x.transform(X_test)

#MinMaxScaler
minmax_scaler_x = MinMaxScaler()
minmax_scaler_x.fit(X_train)
X_train_minmax_scaler_transformed = minmax_scaler_x.transform(X_train)
X_test_minmax_scaler_transformed = minmax_scaler_x.transform(X_test)

params = [10**i for i in range(-5, 6)]

# RidgeCV
model_ridge_standart_scaler_cv = RidgeCV(alphas = params)
model_ridge_standart_scaler_cv.fit(X_train_standart_scaler_transformed, y_train)
ridge_standart_scaler_cv_pred = model_ridge_standart_scaler_cv.predict(X_test_standart_scaler_transformed)

model_ridge_min_max_scaler_cv = RidgeCV(alphas = params)
model_ridge_min_max_scaler_cv.fit(X_train_minmax_scaler_transformed, y_train)
ridge_min_max_scaler_cv_pred = model_ridge_min_max_scaler_cv.predict(X_test_minmax_scaler_transformed)

# LassoCV
model_lasso_standart_scaler_cv = LassoCV(alphas = params)
model_lasso_standart_scaler_cv.fit(X_train_standart_scaler_transformed, y_train)
lasso_standart_scaler_cv_pred = model_lasso_standart_scaler_cv.predict(X_test_standart_scaler_transformed)

model_lasso_min_max_scaler_cv = LassoCV(alphas = params)
model_lasso_min_max_scaler_cv.fit(X_train_minmax_scaler_transformed, y_train)
lasso_min_max_scaler_cv_pred = model_lasso_min_max_scaler_cv.predict(X_test_minmax_scaler_transformed)

r2_ridge_standart_scaler_cv = r2_score(y_test, ridge_standart_scaler_cv_pred)
r2_ridge_min_max_scaler_cv = r2_score(y_test, ridge_min_max_scaler_cv_pred)
r2_lasso_standart_scaler_cv = r2_score(y_test, lasso_standart_scaler_cv_pred)
r2_lasso_min_max_scaler_cv = r2_score(y_test, lasso_min_max_scaler_cv_pred)
results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]

r2_ridge_standart_scaler_cv, r2_ridge_min_max_scaler_cv, r2_lasso_standart_scaler_cv, r2_lasso_min_max_scaler_cv

(0.7011546134235593, 0.703488882462826, 0.7027330685181044, 0.703493909097359)

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [9]:
### Ваш код ###
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

#PolynomialFeatures
polynomial_converter = PolynomialFeatures(degree=2,include_bias=False)
poly_features = polynomial_converter.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.2, random_state=101)

#StandardScaler
standart_scaler_x = StandardScaler()
standart_scaler_x.fit(X_train)
X_train_standart_scaler_transformed = standart_scaler_x.transform(X_train)
X_test_standart_scaler_transformed = standart_scaler_x.transform(X_test)

#MinMaxScaler
minmax_scaler_x = MinMaxScaler()
minmax_scaler_x.fit(X_train)
X_train_minmax_scaler_transformed = minmax_scaler_x.transform(X_train)
X_test_minmax_scaler_transformed = minmax_scaler_x.transform(X_test)

# Ridge
model_ridge_standart_scaler = Ridge()
model_ridge_standart_scaler.fit(X_train_standart_scaler_transformed,y_train)
ridge_standart_scaler_pred = model_ridge_standart_scaler.predict(X_test_standart_scaler_transformed)

model_ridge_minmax_scaler = Ridge()
model_ridge_minmax_scaler.fit(X_train_minmax_scaler_transformed,y_train)
ridge_minmax_scaler_pred = model_ridge_minmax_scaler.predict(X_test_minmax_scaler_transformed)

# Lasso
model_lasso_standart_scaler = Lasso()
model_lasso_standart_scaler.fit(X_train_standart_scaler_transformed,y_train)
lasso_standart_scaler_pred = model_lasso_standart_scaler.predict(X_test_standart_scaler_transformed)

model_lasso_minmax_scaler = Lasso()
model_lasso_minmax_scaler.fit(X_train_minmax_scaler_transformed,y_train)
lasso_minmax_scaler_pred = model_lasso_minmax_scaler.predict(X_test_minmax_scaler_transformed)

r2_ridge_standart_scaler_poly = r2_score(y_test, ridge_standart_scaler_pred)
r2_ridge_min_max_scaler_poly = r2_score(y_test, ridge_minmax_scaler_pred)
r2_lasso_standart_scaler_poly = r2_score(y_test, lasso_standart_scaler_pred)
r2_lasso_min_max_scaler_poly = r2_score(y_test, lasso_minmax_scaler_pred)

results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]

r2_ridge_standart_scaler_poly, r2_ridge_min_max_scaler_poly, r2_lasso_standart_scaler_poly, r2_lasso_min_max_scaler_poly

(0.8181752651058332,
 0.7966345636020615,
 0.6807930537311699,
 0.24121621257587678)

7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [10]:
### Ваш код ###
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

#PolynomialFeatures
polynomial_converter = PolynomialFeatures(degree=2,include_bias=False)
poly_features = polynomial_converter.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.2, random_state=101)

#StandardScaler
standart_scaler_x = StandardScaler()
standart_scaler_x.fit(X_train)
X_train_standart_scaler_transformed = standart_scaler_x.transform(X_train)
X_test_standart_scaler_transformed = standart_scaler_x.transform(X_test)

#MinMaxScaler
minmax_scaler_x = MinMaxScaler()
minmax_scaler_x.fit(X_train)
X_train_minmax_scaler_transformed = minmax_scaler_x.transform(X_train)
X_test_minmax_scaler_transformed = minmax_scaler_x.transform(X_test)

params = [10**i for i in range(-5, 6)]

# RidgeCV
model_ridge_standart_scaler_cv = RidgeCV(alphas = params)
model_ridge_standart_scaler_cv.fit(X_train_standart_scaler_transformed, y_train)
ridge_standart_scaler_poly_cv_pred = model_ridge_standart_scaler_cv.predict(X_test_standart_scaler_transformed)

model_ridge_min_max_scaler_cv = RidgeCV(alphas = params)
model_ridge_min_max_scaler_cv.fit(X_train_minmax_scaler_transformed, y_train)
ridge_min_max_scaler_poly_cv_pred = model_ridge_min_max_scaler_cv.predict(X_test_minmax_scaler_transformed)

# LassoCV
model_lasso_standart_scaler_cv = LassoCV(alphas = params, max_iter=1000000000)
model_lasso_standart_scaler_cv.fit(X_train_standart_scaler_transformed, y_train)
lasso_standart_scaler_poly_cv_pred = model_lasso_standart_scaler_cv.predict(X_test_standart_scaler_transformed)

model_lasso_min_max_scaler_cv = LassoCV(alphas = params, max_iter=1000000000)
model_lasso_min_max_scaler_cv.fit(X_train_minmax_scaler_transformed, y_train)
lasso_min_max_scaler_poly_cv_pred = model_lasso_min_max_scaler_cv.predict(X_test_minmax_scaler_transformed)

r2_ridge_standart_scaler_poly_cv = r2_score(y_test, ridge_standart_scaler_poly_cv_pred)
r2_ridge_min_max_scaler_poly_cv = r2_score(y_test, ridge_min_max_scaler_poly_cv_pred)
r2_lasso_standart_scaler_poly_cv = r2_score(y_test, lasso_standart_scaler_poly_cv_pred)
r2_lasso_min_max_scaler_poly_cv = r2_score(y_test, lasso_min_max_scaler_poly_cv_pred)

results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]

r2_ridge_standart_scaler_poly_cv, r2_ridge_min_max_scaler_poly_cv, r2_lasso_standart_scaler_poly_cv, r2_lasso_min_max_scaler_poly_cv

(0.8181752651054464,
 0.8258914935691352,
 0.8304299177817663,
 0.8134437022886349)

8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [12]:
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([('imputer', StandardScaler()),
                     ('polynomial', PolynomialFeatures()),
                     ('regression', Lasso())])
parameters = {'imputer':[StandardScaler(), MinMaxScaler()],
              'polynomial__degree':np.arange(1,6,1),
              'regression':[Lasso(), Ridge()],
              'regression__alpha':np.logspace(-5, 5, 11)}
select_model = GridSearchCV(pipeline, parameters, scoring='r2')
select_model.fit(X_train, y_train)
select_model.best_estimator_

D:\DS\anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.030e+03, tolerance: 2.367e+00
  model = cd_fast.enet_coordinate_descent(
D:\DS\anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.861e+02, tolerance: 2.425e+00
  model = cd_fast.enet_coordinate_descent(
D:\DS\anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.022e+03, tolerance: 2.655e+00
  model = cd_fast

In [ ]:
print('Параметры лучшей модели:\n', select_model.best_params_)
r2_best_model = r2_score(y_test, select_model.predict(X_test))
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

In [14]:
results_regression

,model,task,R2
0,LR,task2,0.703496
1,Ridge,task2,0.698535
2,Lasso,task2,0.654746
3,Ridge_GridSearchCV,task3,0.712425
4,RidgeCV,task3,0.703429
5,Lasso_GridSearchCV,task3,0.712350
6,LassoCV,task3,0.703495
7,Ridge_StandardScaler,task4,0.703263
8,Ridge_MinMaxScaler,task4,0.701020
9,Lasso_StandardScaler,task4,0.627705


http://archive.ics.uci.edu/ml/datasets/Adult

In [3]:
data = pd.read_csv('D:\\DS\\DS school\\adult')
data.head()

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [4]:
### Ваш код ###
data.columns = data.columns.str.replace(' ', '')
data = data.replace(r'\s+','',regex=True)
data['<=50K'].unique()
data.loc[(data['<=50K'] == '<=50K'), 'target'] = 1
data['target'] = data['target'].fillna(0)
data['target'].unique()
X = data.drop(['target', '<=50K'],axis=1)
y = data['target']

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [110]:
y.value_counts()

target
1.0    24719
0.0     7841
Name: count, dtype: int64

In [5]:
### Ваш код ###
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score

Dummy = DummyClassifier(strategy="most_frequent")
Dummy.fit(X, y)

f1_most_frequent = f1_score(y, Dummy.predict(X))
acc_most_frequent = accuracy_score(y, Dummy.predict(X))
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]

f1_most_frequent, acc_most_frequent

(0.8631086436564884, 0.7591830466830467)

11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [112]:
data.info()
#пропусков не обнаружено

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32560 entries, 0 to 32559
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   39             32560 non-null  int64  
 1   State-gov      32560 non-null  object 
 2   77516          32560 non-null  int64  
 3   Bachelors      32560 non-null  object 
 4   13             32560 non-null  int64  
 5   Never-married  32560 non-null  object 
 6   Adm-clerical   32560 non-null  object 
 7   Not-in-family  32560 non-null  object 
 8   White          32560 non-null  object 
 9   Male           32560 non-null  object 
 10  2174           32560 non-null  int64  
 11  0              32560 non-null  int64  
 12  40             32560 non-null  int64  
 13  United-States  32560 non-null  object 
 14  <=50K          32560 non-null  object 
 15  target         32560 non-null  float64
dtypes: float64(1), int64(6), object(9)
memory usage: 4.0+ MB


12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [7]:
### Ваш код ###
data_category = X.select_dtypes('object').columns
data_num = X.select_dtypes('number').columns
X[data_num]

,39,77516,13,2174,0,40
0,50,83311,13,0,0,13
1,38,215646,9,0,0,40
2,53,234721,7,0,0,40
3,28,338409,13,0,0,40
4,37,284582,14,0,0,40
...,...,...,...,...,...,...
32555,27,257302,12,0,0,38
32556,40,154374,9,0,0,40
32557,58,151910,9,0,0,40
32558,22,201490,9,0,0,20


In [11]:
X[data_category]

,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
0,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
1,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
2,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
3,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba
4,Private,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,United-States
...,...,...,...,...,...,...,...,...
32555,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,United-States
32556,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States
32557,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,United-States
32558,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,United-States


13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [12]:
X[data_category].describe(include='object')

,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
count,32560,32560,32560,32560,32560,32560,32560,32560
unique,9,16,7,15,6,5,2,42
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
freq,22696,10501,14976,4140,13193,27815,21789,29169


In [8]:
for col in data_category:
    one_hot = pd.get_dummies(X[col], prefix=col, drop_first=True, dtype=int)
    X = pd.concat((X.drop(col, axis=1), one_hot), axis=1)

In [9]:
X

,39,77516,13,2174,0,40,State-gov_Federal-gov,State-gov_Local-gov,State-gov_Never-worked,State-gov_Private,...,United-States_Portugal,United-States_Puerto-Rico,United-States_Scotland,United-States_South,United-States_Taiwan,United-States_Thailand,United-States_Trinadad&Tobago,United-States_United-States,United-States_Vietnam,United-States_Yugoslavia
0,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,215646,9,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,53,234721,7,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,28,338409,13,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,37,284582,14,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,257302,12,0,0,38,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
32556,40,154374,9,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
32557,58,151910,9,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
32558,22,201490,9,0,0,20,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [10]:
from sklearn.preprocessing import MinMaxScaler
min_max = MinMaxScaler()
X[data_num] = min_max.fit_transform(X[data_num])
X = pd.concat((X.drop(data_num, axis=1), X[data_num]), axis=1)

In [20]:
X

,State-gov_Federal-gov,State-gov_Local-gov,State-gov_Never-worked,State-gov_Private,State-gov_Self-emp-inc,State-gov_Self-emp-not-inc,State-gov_State-gov,State-gov_Without-pay,Bachelors_11th,Bachelors_12th,...,United-States_Trinadad&Tobago,United-States_United-States,United-States_Vietnam,United-States_Yugoslavia,39,77516,13,2174,0,40
0,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0.452055,0.048238,0.800000,0.000000,0.0,0.122449
1,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.287671,0.138113,0.533333,0.000000,0.0,0.397959
2,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0.493151,0.151068,0.400000,0.000000,0.0,0.397959
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0.150685,0.221488,0.800000,0.000000,0.0,0.397959
4,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.273973,0.184932,0.866667,0.000000,0.0,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.136986,0.166404,0.733333,0.000000,0.0,0.377551
32556,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.315068,0.096500,0.533333,0.000000,0.0,0.397959
32557,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.561644,0.094827,0.533333,0.000000,0.0,0.397959
32558,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.068493,0.128499,0.533333,0.000000,0.0,0.193878


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=101, test_size=0.2)

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

In [23]:
X_test

,State-gov_Federal-gov,State-gov_Local-gov,State-gov_Never-worked,State-gov_Private,State-gov_Self-emp-inc,State-gov_Self-emp-not-inc,State-gov_State-gov,State-gov_Without-pay,Bachelors_11th,Bachelors_12th,...,United-States_Trinadad&Tobago,United-States_United-States,United-States_Vietnam,United-States_Yugoslavia,39,77516,13,2174,0,40
22356,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.465753,0.140798,0.533333,0.000000,0.0,0.397959
26008,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0.027397,0.129742,0.400000,0.000000,0.0,0.295918
20733,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.315068,0.024189,0.533333,0.000000,0.0,0.397959
14475,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0.383562,0.136755,0.800000,0.072981,0.0,0.397959
27907,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0.602740,0.100200,0.200000,0.076881,0.0,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21030,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.068493,0.123434,0.600000,0.000000,0.0,0.142857
17105,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.082192,0.134123,0.533333,0.000000,0.0,0.397959
11372,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0.369863,0.090433,0.800000,0.072981,0.0,0.479592
17761,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0.013699,0.072372,0.533333,0.000000,0.0,0.234694


In [13]:
#LogisticRegression
Log_Reg = LogisticRegression(max_iter=1000)
Log_Reg.fit(X_train, y_train)
Log_Reg_pred = Log_Reg.predict(X_test)
Log_Reg_acc =cross_val_score(Log_Reg, X_test, y_test, scoring='accuracy')
Log_Reg_f1 = cross_val_score(Log_Reg, X_test, y_test, scoring='f1')
Log_Reg_acc.mean(), Log_Reg_f1.mean()

(0.8438261933644797, 0.8998314222875334)

In [53]:
# KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=2)
KNN.fit(X_train, y_train)
KNN_pred = KNN.predict(X_test.values)
KNN_acc = cross_val_score(KNN, X_test.values, y_test.values, scoring='accuracy')
KNN_f1 = cross_val_score(KNN, X_test.values, y_test.values, scoring='f1')
KNN_acc.mean(), KNN_f1.mean()

(0.77349517184142, 0.839066552182485)

In [15]:
#LinearSVC
L_SVC = LinearSVC()
L_SVC.fit(X_train, y_train)
L_SVC_pred = L_SVC.predict(X_test)
L_SVC_acc =cross_val_score(L_SVC, X_test, y_test, scoring='accuracy')
L_SVC_f1 = cross_val_score(L_SVC, X_test, y_test, scoring='f1')
L_SVC_acc.mean(), L_SVC_f1.mean()

D:\DS\anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\DS\anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\DS\anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\DS\anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\DS\anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `du

(0.8507756286274052, 0.9048387601589536)

In [54]:
### Ваш код ###
f1_LR = Log_Reg_f1.mean()
acc_LR = Log_Reg_acc.mean()
f1_KNN = KNN_f1.mean()
acc_KNN = KNN_acc.mean()
f1_SVM = L_SVC_f1.mean()
acc_SVM = L_SVC_acc.mean()
results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [42]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

preprocessor_imputer = ColumnTransformer([('cat', Pipeline([('imputer', SimpleImputer(missing_values='?', 
                                                                                      strategy="most_frequent")), 
                                                            ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first',
                                                                                      sparse=False))]), data_category),
                                          ('num', Pipeline([('scaler', MinMaxScaler())]), data_num)])

In [43]:
X_SI = data.drop(['target', '<=50K'],axis=1)
X_SI

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States
32556,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
32557,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States
32558,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States


In [44]:
X_SI = pd.DataFrame(preprocessor_imputer.fit_transform(X_SI))
X_SI

D:\DS\anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.452055,0.048238,0.800000,0.000000,0.0,0.122449
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.287671,0.138113,0.533333,0.000000,0.0,0.397959
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.493151,0.151068,0.400000,0.000000,0.0,0.397959
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.150685,0.221488,0.800000,0.000000,0.0,0.397959
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.273973,0.184932,0.866667,0.000000,0.0,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.136986,0.166404,0.733333,0.000000,0.0,0.377551
32556,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.315068,0.096500,0.533333,0.000000,0.0,0.397959
32557,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.561644,0.094827,0.533333,0.000000,0.0,0.397959
32558,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.068493,0.128499,0.533333,0.000000,0.0,0.193878


In [45]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

X_train, X_test, y_train, y_test = train_test_split(X_SI, y, random_state=101, test_size=0.2)

In [51]:
#LogisticRegression
Log_Reg_SI = LogisticRegression(max_iter=1000)
Log_Reg_SI.fit(X_train, y_train)
Log_Reg_SI_pred = Log_Reg_SI.predict(X_test)
Log_Reg_SI_acc =cross_val_score(Log_Reg_SI, X_test, y_test, scoring='accuracy')
Log_Reg_SI_f1 = cross_val_score(Log_Reg_SI, X_test, y_test, scoring='f1')
Log_Reg_SI_acc.mean(), Log_Reg_SI_f1.mean()

(0.8439795674167966, 0.899995751655414)

In [52]:
# KNeighborsClassifier
KNN_SI = KNeighborsClassifier(n_neighbors=2)
KNN_SI.fit(X_train, y_train)
KNN_SI_pred = KNN_SI.predict(X_test.values)
KNN_SI_acc =cross_val_score(KNN_SI, X_test.values, y_test.values, scoring='accuracy')
KNN_SI_f1 = cross_val_score(KNN_SI, X_test.values, y_test.values, scoring='f1')
KNN_SI_acc.mean(), KNN_SI_f1.mean()

(0.77349517184142, 0.839066552182485)

In [55]:
#LinearSVC
L_SVC_SI = LinearSVC()
L_SVC_SI.fit(X_train, y_train)
L_SVC_SI_pred = L_SVC_SI.predict(X_test)
L_SVC_SI_acc =cross_val_score(L_SVC_SI, X_test, y_test, scoring='accuracy')
L_SVC_SI_f1 = cross_val_score(L_SVC_SI, X_test, y_test, scoring='f1')
L_SVC_SI_acc.mean(), L_SVC_SI_f1.mean()

D:\DS\anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\DS\anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\DS\anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\DS\anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\DS\anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `du

(0.8499309957039511, 0.9044544408352791)

In [56]:
f1_LR = Log_Reg_SI_f1.mean()
acc_LR = Log_Reg_SI_acc.mean()
f1_KNN = KNN_SI_f1.mean()
acc_KNN = KNN_SI_acc.mean()
f1_SVM = L_SVC_SI_f1.mean()
acc_SVM = L_SVC_SI_acc.mean()
results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]

15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [78]:
data_del = data.copy()
data_del

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,target
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,1.0
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,1.0
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,1.0
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,1.0
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,1.0
32556,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,0.0
32557,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,1.0
32558,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K,1.0


In [79]:
data_del = data_del.replace({'?': np.nan}).dropna()
data_del

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,target
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,1.0
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,1.0
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,1.0
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,1.0
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,1.0
32556,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,0.0
32557,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,1.0
32558,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K,1.0


In [80]:
X_del = data_del.drop(['target', '<=50K'],axis=1)
y_del = data_del['target']
data_category = X_del.select_dtypes('object').columns
data_num = X_del.select_dtypes('number').columns
X_del.reset_index(drop=True, inplace=True)
y_del.reset_index(drop=True, inplace=True)

In [81]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler

preprocessor_imputer = ColumnTransformer([('cat', Pipeline([('encoder', OneHotEncoder(handle_unknown='ignore', drop='first',
                                                                                      sparse=False))]), data_category),
                                          ('num', Pipeline([('scaler', MinMaxScaler())]), data_num)])

X_del = pd.DataFrame(preprocessor_imputer.fit_transform(X_del))
X_del

D:\DS\anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.452055,0.047277,0.800000,0.000000,0.0,0.122449
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.287671,0.137244,0.533333,0.000000,0.0,0.397959
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.493151,0.150212,0.400000,0.000000,0.0,0.397959
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.150685,0.220703,0.800000,0.000000,0.0,0.397959
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.273973,0.184109,0.866667,0.000000,0.0,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30156,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.136986,0.165563,0.733333,0.000000,0.0,0.377551
30157,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.315068,0.095589,0.533333,0.000000,0.0,0.397959
30158,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.561644,0.093914,0.533333,0.000000,0.0,0.397959
30159,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.068493,0.127620,0.533333,0.000000,0.0,0.193878


In [82]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

X_train, X_test, y_train, y_test = train_test_split(X_del, y_del, random_state=101, test_size=0.2)

In [83]:
#LogisticRegression
Log_Reg_del = LogisticRegression(max_iter=1000)
Log_Reg_del.fit(X_train, y_train)
Log_Reg_del_pred = Log_Reg_del.predict(X_test)
Log_Reg_del_acc =cross_val_score(Log_Reg_del, X_test, y_test, scoring='accuracy')
Log_Reg_del_f1 = cross_val_score(Log_Reg_del, X_test, y_test, scoring='f1')
Log_Reg_del_acc.mean(), Log_Reg_del_f1.mean()

(0.8372282470552512, 0.8939433699101398)

In [84]:
# KNeighborsClassifier
KNN_del = KNeighborsClassifier(n_neighbors=2)
KNN_del.fit(X_train, y_train)
KNN_del_pred = KNN_del.predict(X_test.values)
KNN_del_acc =cross_val_score(KNN_del, X_test.values, y_test.values, scoring='accuracy')
KNN_del_f1 = cross_val_score(KNN_del, X_test.values, y_test.values, scoring='f1')
KNN_del_acc.mean(), KNN_del_f1.mean()

(0.747222600062378, 0.8163110009204605)

In [85]:
#LinearSVC
L_SVC_del = LinearSVC()
L_SVC_del.fit(X_train, y_train)
L_SVC_del_pred = L_SVC_del.predict(X_test)
L_SVC_del_acc =cross_val_score(L_SVC_del, X_test, y_test, scoring='accuracy')
L_SVC_del_f1 = cross_val_score(L_SVC_del, X_test, y_test, scoring='f1')
L_SVC_del_acc.mean(), L_SVC_del_f1.mean()

D:\DS\anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\DS\anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\DS\anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\DS\anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\DS\anaconda\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `du

(0.8469829614630596, 0.9015877198586605)

In [86]:
f1_LR_del_missings = Log_Reg_del_f1.mean()
acc_LR_del_missings = Log_Reg_del_acc.mean()
f1_KNN_del_missings = KNN_del_f1.mean()
acc_KNN_del_missings = KNN_del_acc.mean()
f1_SVM_del_missings = L_SVC_del_f1.mean()
acc_SVM_del_missings = L_SVC_del_acc.mean()
results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [92]:
data_SI = data.copy()
data_SI

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,target
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,1.0
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,1.0
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,1.0
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,1.0
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,1.0
32556,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,0.0
32557,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,1.0
32558,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K,1.0


In [93]:
X_SI = data_SI.drop(['target', '<=50K'],axis=1)
y_SI = data_SI['target']
data_category = X_SI.select_dtypes('object').columns
data_num = X_SI.select_dtypes('number').columns

In [94]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

preprocessor_imputer = ColumnTransformer([('cat', Pipeline([('imputer', SimpleImputer(missing_values='?', 
                                                                                      strategy="most_frequent")), 
                                                            ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first',
                                                                                      sparse=False))]), data_category),
                                          ('num', Pipeline([('scaler', MinMaxScaler())]), data_num)])

In [95]:
X_SI = pd.DataFrame(preprocessor_imputer.fit_transform(X_SI))
X_SI

D:\DS\anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.452055,0.048238,0.800000,0.000000,0.0,0.122449
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.287671,0.138113,0.533333,0.000000,0.0,0.397959
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.493151,0.151068,0.400000,0.000000,0.0,0.397959
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.150685,0.221488,0.800000,0.000000,0.0,0.397959
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.273973,0.184932,0.866667,0.000000,0.0,0.397959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.136986,0.166404,0.733333,0.000000,0.0,0.377551
32556,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.315068,0.096500,0.533333,0.000000,0.0,0.397959
32557,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.561644,0.094827,0.533333,0.000000,0.0,0.397959
32558,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.068493,0.128499,0.533333,0.000000,0.0,0.193878


In [97]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

X_train, X_test, y_train, y_test = train_test_split(X_SI, y, random_state=101, test_size=0.2)

In [104]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# RandomForestClassifier
RF = RandomForestClassifier(max_depth=2, random_state=0).fit(X_train, y_train)
RF_pred = RF.predict(X_test)
acc_RF = cross_val_score(RF, X_test, y_test, cv=3, scoring='accuracy')
f1_RF = cross_val_score(RF, X_test, y_test, cv=3, scoring='f1')
acc_RF.mean(), f1_RF.mean()

(0.7684273424084126, 0.8674653904171877)

In [105]:
# GradientBoostingClassifier
GB = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
GB_pred = GB.predict(X_test)
acc_GB = cross_val_score(GB, X_test, y_test, cv=3, scoring='accuracy')
f1_GB = cross_val_score(GB, X_test, y_test, cv=3, scoring='f1')
acc_GB.mean(), f1_GB.mean()

(0.8581073938616918, 0.908886723864904)

In [106]:
f1_RF = f1_RF.mean()
acc_RF = acc_RF.mean()
f1_GB = f1_GB.mean()
acc_GB = acc_GB.mean()
results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [ ]:
preprocessor_imputer = ColumnTransformer([('cat', Pipeline([('encoder', OneHotEncoder(handle_unknown='ignore', 
                                                                                      sparse=False))]), categorical),
                                                            ('num', Pipeline([('scaler', MinMaxScaler())]), numerical)])

encoders = [OneHotEncoder(handle_unknown='ignore'), LabelEncoder()]
scalers = [StandardScaler(), MinMaxScaler()]
models = [LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
          SVC(random_state=RANDOM_STATE),
          LinearSVC(random_state=RANDOM_STATE),
          RandomForestClassifier(random_state=RANDOM_STATE),
          GradientBoostingClassifier(random_state=RANDOM_STATE)]

search_model = Pipeline(steps=[('preprocessor', preprocessor_imputer), 
                               ('classifier', LogisticRegression(random_state=RANDOM_STATE, max_iter=1000))])

params = {
          'preprocessor__cat__encoder': encoders,
          'preprocessor__num': scalers,
          'classifier': models}

BestModel = GridSearchCV(search_model,
                                 param_grid=params,
                                 scoring=('accuracy', 'f1'),
                                 refit='f1')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=RANDOM_STATE)
BestModel.fit(X_test, y_test)
BestModel_acc = cross_val_score(BestModel, X_test, y_test, cv=3, scoring='accuracy')
BestModel_f1 = cross_val_score(BestModel, X_test, y_test, cv=3, scoring='f1')

In [ ]:
best_params = BestModel.best_params_
print('Параметры лучшей модели:\n', best_params)
f1_best = BestModel_f1.mean()
acc_best = BestModel_acc.mean()
results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

In [107]:
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.863109,0.759183
7,LogisticRegression_delete_missings,task15,0.893943,0.837228
8,KNeighborsClassifier_delete_missings,task15,0.816311,0.747223
9,LinearSVC_delete_missings,task15,0.901588,0.846983
10,RandomForestClassifier,task16,0.867465,0.768427
11,GradientBoostingClassifier,task16,0.908887,0.858107
